# Descarga de registros de presencia de especies de GBIF

Este [Jupyter Notebook](https://jupyter.org/), desarrollado en el lenguaje de programación [Python](https://www.python.org/), genera un archivo para descargar del [portal de GBIF](https://www.gbif.org/) con los registros de presencia de las especies contenidas en un archivo CSV.

**Entradas**
- Un archivo CSV con nombres científicos de especies.
- Las especificadas en la sección de **Constantes**.

**Procesamiento**
- Se obtienen las llaves (*keys*) de las especies en el [API de GBIF](https://www.gbif.org/developer/summary).
- En caso de ser necesario, se filtran las llaves (ej. se conservan solo las de nombres que tengan coincidencia exacta o solo las de nombres aceptados).
- Con base en la lista de llaves, se construye una consulta para el [portal de GBIF](https://www.gbif.org/).
- El archivo CSV retornado por el portal se descarga y se renombra con el valor de la constante OCCURRENCES_CSV.
- Se recorre el archivo CSV para generar las salidas que se describen seguidamente.

**Salidas**
- Un archivo con los registros de presencia de todas las especies para descargar del portal de GBIF.
- Para cada especie en la lista de entrada, se genera:
  - Un archivo CSV con los registros de presencia.

**Bibliotecas de Python**

In [ ]:
import pandas as pd

# El siguiente archivo debe estar en el mismo directorio que este notebook
from functions_query_from_species_list import *

**Constantes**

In [ ]:
# Credenciales para el API de GBIF
GBIF_USER_NAME = "<USUARIO>"
GBIF_PASSWORD = "<CLAVE>"
GBIF_NOTIFICATION_ADDRESSES = "email"
GBIF_DOWNLOAD_FORMAT = "SIMPLE_CSV"

# Directorio de entrada
INPUT_DIR = "C:/Users/mfvargas/descarga-registros-presencia-gbif/"

# Archivo CSV de entrada con lista de especies a procesar
INPUT_CHECKLIST = INPUT_DIR + "Anexo 1. Listado de especies gradiente .csv"
# Columna con el nombre científico de la especie
INPUT_SCINAME_COL = "NOMBRE_COMPLETO"

# Directorio de salida
OUTPUT_DIR = "C:/Users/mfvargas/descarga-registros-presencia-gbif/salidas/"

# Archivo CSV con especies del archivo de entrada que se procesan
INPUT_CHECKLIST_PROCESSED_SPECIES = OUTPUT_DIR + "especies-procesadas.csv"

# Archivo CSV con especies del archivo de entrada que no se procesan
INPUT_CHECKLIST_NON_PROCESSED_SPECIES = OUTPUT_DIR + "especies-no-procesadas.csv"

# Archivo CSV de salida con los registros de presencia de todas las especies
OCCURRENCES_CSV = INPUT_DIR + "registros-presencia-todos.csv"

# Número máximo de registros a desplegar en los dataframes de Pandas
pd.options.display.max_rows = 15

## Carga de datos

In [ ]:
# Carga del archivo CSV de entrada en un dataframe de Pandas
input_species_df = pd.read_csv(INPUT_CHECKLIST, encoding='utf_8')

input_species_df

Se consulta el API de GBIF para obtener las llaves (*keys*) de las especies

In [ ]:
# Se obtienen las llaves de las especies a través del API de GBIF
gbif_species_df = match_species(input_species_df, INPUT_SCINAME_COL)

gbif_species_df

In [ ]:
# Se construye una lista de nombres que van a procesarse
gbif_species_processed_df = gbif_species_df.loc[(gbif_species_df["matchType"]=="EXACT")]
gbif_species_processed_df.to_csv(INPUT_CHECKLIST_PROCESSED_SPECIES)

gbif_species_processed_df

In [ ]:
# Se construye una lista de nombres que no van a procesarse
gbif_species_non_processed_df = gbif_species_df.loc[~((gbif_species_df["matchType"]=="EXACT"))]
gbif_species_non_processed_df.to_csv(INPUT_CHECKLIST_NON_PROCESSED_SPECIES)

gbif_species_non_processed_df

In [ ]:
# Se separa la lista de llaves
key_list = gbif_species_processed_df.usageKey.tolist()

key_list

Se construye una consulta para descarga en el portal de GBIF

In [ ]:
# Se construye una consulta para descarga en el portal de GBIF
download_query = {}
download_query["creator"] = GBIF_USER_NAME
download_query["notificationAddresses"] = [GBIF_NOTIFICATION_ADDRESSES]
download_query["sendNotification"] = True
download_query["format"] = GBIF_DOWNLOAD_FORMAT
download_query["predicate"] =   {"type":"and", "predicates": 
                                 [
                                    {"type":"equals", "key":"HAS_COORDINATE",       "value":"true"},
                                    {"type":"equals", "key":"HAS_GEOSPATIAL_ISSUE", "value":"false"}, 
                                    {"type":"in",     "key": "TAXON_KEY",           "values":key_list},
                                    {
                                      "type": "or",
                                      "predicates": [
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "PA"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "CR"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "NI"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "HN"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "SV"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "GT"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "BZ"
                                        },
                                        {
                                          "type": "equals",
                                          "key": "COUNTRY",
                                          "value": "MX"
                                        }
                                      ]
                                    }                                     
                                 ]
                                }

download_query

In [ ]:
# Consulta para el API de GBIF
create_download_given_query(GBIF_USER_NAME, GBIF_PASSWORD, download_query)

# Respuesta esperada:
# ok
# <Response [201]>

La consulta al API genera un archivo ZIP para descarga en
[https://www.gbif.org/user/download](https://www.gbif.org/user/download)

**ESTE ARCHIVO ZIP DEBE DESCOMPRIMIRSE Y RENOMBRARSE CON EL NOMBRE ESPECIFICADO EN LA CONSTANTE "OCCURRENCES_CSV"**

In [ ]:
occurrences_df = pd.read_csv(OCCURRENCES_CSV, sep='\t')

occurrences_df

In [ ]:
# ==================== RECORRIDO DE LA LISTA DE ESPECIES ====================
for index, row in gbif_species_processed_df.iterrows():
    species_input = row["inputName"]
    species_gbif = row["species"]
    print(species_gbif)

    current_taxon_df = occurrences_df[occurrences_df['species'] == species_gbif]    
    if current_taxon_df.size > 0:   
        # Creación de archivo de registros de presencia de la especie en archivo CSV
        current_taxon_df = current_taxon_df[['basisOfRecord', 'species', 'catalogNumber', 'recordNumber', 'decimalLatitude', 'decimalLongitude', 'locality', 'year', 'recordedBy']]
        current_taxon_df.to_csv(OUTPUT_DIR + "registros-presencia-" + species_input.replace(" ", "_") + ".csv")